## UR5e control with python

Installation guide: https://sdurobotics.gitlab.io/ur_rtde/installation/installation.html

pip install --user ur_rtde

sudo apt-get install libboost-all-dev

git submodule update --init --recursive

https://github.com/UniversalRobots/Universal_Robots_ROS_Driver/blob/master/ur_robot_driver/doc/install_urcap_e_series.md


Initialization

In [20]:
import rtde_control
import rtde_receive
import time
from rtde_control import Path, PathEntry

In [31]:
rtde_c = rtde_control.RTDEControlInterface("192.168.2.1")  # IP address found on robot
rtde_r = rtde_receive.RTDEReceiveInterface("192.168.2.1")

Init Poses 

In [13]:
middle_pos = [
    -0.5892404615419603,
    0.030096186269537212,
    0.0204709361794337,
    -2.2120065495659795,
    -2.205496603905911,
    -0.023032448184135128,
]

middle_joints = [
    3.3211379051208496,
    -2.336783548394674,
    -1.9027351140975952,
    -0.47548242033038335,
    1.5530344247817993,
    0.19170111417770386,
]

home_pos = [
    -0.19161405312765759,
    0.13472399156690068,
    0.365305811889139,
    -2.216389383069919,
    -2.214478541184546,
    -0.011795250688296986,
]

home_joints = [
    3.141732692718506,
    -0.7853349608233948,
    -2.3561296463012695,
    -1.5707891744430817,
    1.570825219154358,
    6.953880983928684e-06,
]

test_pick_pos = [
    -0.7974861406917716,
    0.13477626960710312,
    -0.24014838365129754,
    -2.216511611739429,
    -2.2144969248838295,
    -0.011791118425820252,
]

above_conveyor_pos = [
    -0.10178399097655502,
    0.42530727780286376,
    0.36529650939521296,
    -2.2164100615996616,
    -2.2145170665255822,
    -0.011858118701172806,
]

place_tube_pos = [
    -0.10182622635392878,
    0.42531419687684324,
    0.2327819096662583,
    -2.216210720751703,
    -2.214618424988339,
    -0.01182593627625963,
]

Get current pose (coordinates)

In [14]:
pose = rtde_r.getActualTCPPose()
pose

[-0.1916296911212659,
 0.13505077364298387,
 0.39980321581677863,
 -2.216400369508157,
 -2.2146255574355225,
 -0.011787709943431837]

Get current joint positions

In [15]:
init_q = rtde_r.getActualQ()
init_q

[3.1421308517456055,
 -0.8009743851474305,
 -2.187337875366211,
 -1.7240215740599574,
 1.5705572366714478,
 -0.0011161009417932632]

Move to specified position

In [6]:
pose[0] -= 0.2
pose[2] -= 0.4

In [7]:
rtde_c.moveL(
    pose, 0.1, 0.1
)  # First argument is the pose 6d vector followed by speed and acceleration

False

Move to specified joint positions

In [8]:
rtde_c.moveJ(home_joints, 0.2, 0.2)

RTDEControlInterface: RTDE control script is not running!


False

### Test program

In [15]:
# Move to init
rtde_c.moveJ(home_joints, 0.5, 0.5, True)
time.sleep(3)

# Move to middle pos
rtde_c.moveJ(middle_joints, 0.5, 0.5, True)

True

In [16]:
path1 = Path()
path2 = Path()
vel_L = 0.2
acc_L = 0.2
vel_J = 1.4
acc_J = 1.4
blend = 0.099
path1.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            home_pos[0],
            home_pos[1],
            home_pos[2],
            home_pos[3],
            home_pos[4],
            home_pos[5],
            vel_J,
            acc_J,
            0.0,
        ],
    )
)
path1.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            middle_pos[0],
            middle_pos[1],
            middle_pos[2],
            middle_pos[3],
            middle_pos[4],
            middle_pos[5],
            vel_J,
            acc_J,
            blend,
        ],
    )
)
path1.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            test_pick_pos[0] + 0.3,
            test_pick_pos[1],
            test_pick_pos[2],
            test_pick_pos[3],
            test_pick_pos[4],
            test_pick_pos[5],
            vel_L,
            acc_L,
            0.0,
        ],
    )
)

path2.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            test_pick_pos[0] + 0.3,
            test_pick_pos[1],
            test_pick_pos[2] + 0.2,
            test_pick_pos[3],
            test_pick_pos[4],
            test_pick_pos[5],
            vel_J,
            vel_J,
            blend,
        ],
    )
)
path2.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            home_pos[0],
            home_pos[1],
            home_pos[2],
            home_pos[3],
            home_pos[4],
            home_pos[5],
            vel_J,
            acc_J,
            blend,
        ],
    )
)
path2.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            above_conveyor_pos[0],
            above_conveyor_pos[1],
            above_conveyor_pos[2],
            above_conveyor_pos[3],
            above_conveyor_pos[4],
            above_conveyor_pos[5],
            vel_J,
            acc_J,
            blend,
        ],
    )
)
path2.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            place_tube_pos[0],
            place_tube_pos[1],
            place_tube_pos[2],
            place_tube_pos[3],
            place_tube_pos[4],
            place_tube_pos[5],
            vel_J,
            vel_J,
            0.0,
        ],
    )
)
path2.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            above_conveyor_pos[0],
            above_conveyor_pos[1],
            above_conveyor_pos[2],
            above_conveyor_pos[3],
            above_conveyor_pos[4],
            above_conveyor_pos[5],
            vel_J,
            vel_J,
            blend,
        ],
    )
)
path2.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            home_pos[0],
            home_pos[1],
            home_pos[2],
            home_pos[3],
            home_pos[4],
            home_pos[5],
            vel_J,
            acc_J,
            0.0,
        ],
    )
)

In [17]:
rtde_c.movePath(path1, False)
time.sleep(3)
rtde_c.movePath(path2, False)

Digital Output Control

In [17]:
import rtde_io

rtde_io = rtde_io.RTDEIOInterface("192.168.2.1")

In [32]:
rtde_io.setStandardDigitalOut(0, True)

True

In [22]:
home_pos = [
    -0.19163239551678288,
    0.13504921721798657,
    0.3352908442361863,
    -2.216415003649968,
    -2.214597896269351,
    -0.011771235355496306,
]


middle_pos = [
    -0.5892404615419603,
    0.030096186269537212,
    0.0204709361794337,
    -2.2120065495659795,
    -2.205496603905911,
    -0.023032448184135128,
]

above_conveyor_pos = [
    -0.10178399097655502,
    0.42530727780286376,
    0.36529650939521296,
    -2.2164100615996616,
    -2.2145170665255822,
    -0.011858118701172806,
]

place_tube_pos = [
    -0.09530795655436443,
    0.4193777515105637,
    0.3130236559091107,
    -2.2187837262553156,
    -2.209625346523522,
    0.0005215327962423823,
]

In [23]:
rtde_c.moveL(home_pos, 0.2, 0.2)

False

In [44]:
path = Path()
blend = 0.0
vel_J = 0.2
acc_J = 0.2
path.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            home_pos[0],
            home_pos[1],
            home_pos[2],
            home_pos[3],
            home_pos[4],
            home_pos[5],
            vel_J,
            acc_J,
            blend,
        ],
    )
)
path.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            middle_pos[0],
            middle_pos[1],
            middle_pos[2],
            middle_pos[3],
            middle_pos[4],
            middle_pos[5],
            vel_J,
            acc_J,
            blend,
        ],
    )
)
path.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            home_pos[0],
            home_pos[1],
            home_pos[2],
            home_pos[3],
            home_pos[4],
            home_pos[5],
            vel_J,
            acc_J,
            blend,
        ],
    )
)
path.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            place_tube_pos[0],
            place_tube_pos[1],
            place_tube_pos[2],
            place_tube_pos[3],
            place_tube_pos[4],
            place_tube_pos[5],
            vel_J,
            acc_J,
            blend,
        ],
    )
)

rtde_c.movePath(path, False)

KeyboardInterrupt: 

In [35]:
rtde_c.moveL(middle_pos, 0.2, 0.2)

True

In [1]:
from robotiq_gripper_control import RobotiqGripper

print("Activating Gripper")
gripper = RobotiqGripper(rtde_c)
gripper.activate()  # returns to previous position after activation
gripper.set_force(0)  # from 0 to 100 %
gripper.set_speed(100)  # from 0 to 100 %
print("Gripper activated")

Activating Gripper


NameError: name 'rtde_c' is not defined

In [68]:
path = Path()
gripper.close()
blend = 0.03
vel_J = 1.5
acc_J = 1.5
path = Path()
path.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            home_pos[0],
            home_pos[1],
            home_pos[2],
            home_pos[3],
            home_pos[4],
            home_pos[5],
            vel_J,
            acc_J,
            blend,
        ],
    )
)
path.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            above_conveyor_pos[0],
            above_conveyor_pos[1],
            above_conveyor_pos[2],
            above_conveyor_pos[3],
            above_conveyor_pos[4],
            above_conveyor_pos[5],
            vel_J,
            acc_J,
            blend,
        ],
    )
)
path.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            place_tube_pos[0],
            place_tube_pos[1],
            place_tube_pos[2],
            place_tube_pos[3],
            place_tube_pos[4],
            place_tube_pos[5],
            vel_J,
            acc_J,
            0.0,
        ],
    )
)
rtde_c.movePath(path, False)
gripper.open()
path2 = Path()
path2.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            above_conveyor_pos[0],
            above_conveyor_pos[1],
            above_conveyor_pos[2],
            above_conveyor_pos[3],
            above_conveyor_pos[4],
            above_conveyor_pos[5],
            vel_J,
            acc_J,
            blend,
        ],
    )
)
path2.addEntry(
    PathEntry(
        PathEntry.MoveJ,
        PathEntry.PositionTcpPose,
        [
            home_pos[0],
            home_pos[1],
            home_pos[2],
            home_pos[3],
            home_pos[4],
            home_pos[5],
            vel_J,
            acc_J,
            0.0,
        ],
    )
)
rtde_c.movePath(path2, False)
gripper.close()

True

Exception: Operation canceled
RTDEControlInterface: Could not receive data from robot...
Operation canceled
RTDEControlInterface: Robot is disconnected, reconnecting...


Reconnecting...
Dashboard client deadline expired


RTDEControlInterface: Could not reconnect to robot...
Timeout connecting to UR dashboard server.


In [66]:
gripper.open()

True

In [67]:
gripper.close()

True

In [16]:
import robotiq_gripper

ip = "192.168.2.1"
gripper = robotiq_gripper.RobotiqGripper()
gripper.connect(ip, 63352)

RuntimeError: Calibration failed because of an object: ObjectStatus.STOPPED_INNER_OBJECT

In [10]:
gripper.move(120, speed=100, force=0)  # close gripper before gripping

(True, 120)

In [24]:
gripper.move(140, speed=25, force=0)  # gripping

(True, 140)

In [12]:
gripper.is_closed()

False

In [18]:
gripper.get_current_position()

246

In [19]:
gripper.move(0, speed=100, force=0)

(True, 0)